In [3]:
%pip install ibm_watsonx_ai

  Using cached anyio-4.4.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.6/982.6 kB 26.2 MB/s eta 0:00:00
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached anyio-4.4.0-py3-none-any.whl (86 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import ibm_watsonx_ai
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
import getpass
import os

In [2]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your WML api key (hit enter): ")
}

In [3]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

In [4]:
#not being used yet; waiting on llava model to be released
images = ["https://i2.wp.com/www.statsaustralia.com.au/wp-content/uploads/stats-wa-forensic-pavement-failure-investigation.jpg",
          "https://cdn.shopify.com/s/files/1/0286/3900/2698/products/TVN_Huile-olive-infuse-et-s-227x300_e9a90ffd-b6d2-4118-95a1-29a5c7a05a49_800x.jpg?v=1616684087"]
#idea: create an images and queries dictionary and iterate through the dict items to ask llm multiple questions

In [5]:
#iterate through queries using the iterative prompt
queries = ["what is the capital of Germany?", "Who was Sally Hemings?"]

In [6]:
client = ibm_watsonx_ai.APIClient(
            credentials = ibm_watsonx_ai.Credentials(
                url = credentials["url"],
                api_key =credentials["apikey"],
            ), 
            project_id=project_id
        )

In [7]:
# iterative_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
# Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
# # 
# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don'\''t know the answer to a question, please don'\''t share false information.\ninput: {query}\noutput: '"""

prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
# 
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don'\''t know the answer to a question, please don'\''t share false information.\ninput: What is the capital of Germany?\noutput: '"""


In [8]:
params = {
            GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
            GenParams.MAX_NEW_TOKENS: 200,
            GenParams.MIN_NEW_TOKENS: 1,
        }

In [10]:
model = Model(
            # model_id='meta-llama/llama-3-70b-instruct',
            model_id = 'meta-llama/llama3-llava-next-8b-hf',
            # model_id = 'meta-llama/llama-3-405b-instruct'
            params=params,
            credentials=client.credentials,
            project_id=project_id,
        )

In [11]:
model.generate(prompt="What is this an image of https://a0.muscache.com/im/pictures/dbb5814b-2fae-4d6a-a34d-872d05d98286.jpg?im_w=720")

{'model_id': 'meta-llama/llama3-llava-next-8b-hf',
 'created_at': '2024-09-19T20:58:35.653Z',
 'results': [{'generated_text': "\nThis is an image of a person's face.",
   'generated_token_count': 12,
   'input_token_count': 47,
   'stop_reason': 'eos_token'}],
 'system': {'warnings': [{'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.',
    'id': 'disclaimer_warning',
    'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-models.html?context=wx'}]}}

In [31]:
def generate_prompt(query:str = ''):
    prompt =f"""You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
            Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
            If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don'\''t know the answer to a question, please don'\''t share false information.\ninput: {query}\noutput: '"""
    return prompt

for q in queries:
    p = generate_prompt(q)
    print(p)

You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
            Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
            If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don'''t know the answer to a question, please don'''t share false information.
input: what is the capital of Germany?
output: '
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
            Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
            If a question does not make any sense, or is not factually coherent, explain why instead o

In [34]:
for query in queries:
    print(query)
    p = generate_prompt(query)
    result = model.generate(prompt=p)
    print(result['results'][0]['generated_text'].strip())

what is the capital of Germany?
The capital of Germany is Berlin.'
Who was Sally Hemings?
Sally Hemings was an enslaved woman owned by Thomas Jefferson, the third President of the United States. She was born in 1773 and died in 1835. Hemings was a half-sister to Jefferson's wife, Martha Wayles Jefferson, as they shared the same father, John Wayles. Sally Hemings had six children, and it is widely believed that Thomas Jefferson was their father. This has been supported by DNA evidence and historical research. Hemings lived at Monticello, Jefferson's estate in Virginia, where she worked as a seamstress and domestic servant. Despite her enslavement, Hemings was able to negotiate a degree of freedom for herself and her children, and they were eventually freed in Jefferson's will.'


In [25]:
result['results'][0]['generated_text'].strip()

"The capital of Germany is Berlin.'"

In [26]:
from ibm_watsonx_ai.foundation_models.prompts import PromptTemplate, PromptTemplateManager
# https://ibm.github.io/watsonx-ai-python-sdk/prompt_template_manager.html

In [27]:
prompt_mgr = PromptTemplateManager(
                credentials=ibm_watsonx_ai.Credentials(
                url = credentials["url"],
                api_key =credentials["apikey"],
            ), 
            project_id=project_id
        )

In [13]:
prompt_template_text = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. 
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don'\''t know the answer to a question, please don'\''t share false information.\nQuery: {input_query}'"""

In [18]:
prompt_template = PromptTemplate(name="My prompt",
                                 model_id='meta-llama/llama-3-70b-instruct',
                                 input_prefix="Human:",
                                 output_prefix="Assistant:",
                                 input_text=prompt_template_text,
                                 input_variables=['input_query'],)

In [22]:
stored_prompt_template = prompt_mgr.store_prompt(prompt_template)

In [ ]:
prompt_template